In [49]:
!import csv
from copy import copy
import os
from math import asin,acos,atan,sqrt,sin,cos,tan,pi
import classes
from datetime import datetime
import time
from classes import *
from tools import *
from load import *
import networkx as nx
import itertools
CAPACITY = 140
[ls_site,ls_spot,ls_shop,ls_dorder,ls_otoorder,ls_courier] = LoadAll()
num_dorder = len(ls_dorder)
total_worker = 0
cost = 0
istart = 32
iend = 33

def loadTime(demand):
    return 3*sqrt(demand)+5
[ls_site,ls_spot,ls_shop,ls_dorder,ls_otoorder,ls_courier] = LoadAll()
num_dorder = len(ls_dorder)
total_worker = 0
cost = 0
ls_dtask = []
istart = 0
iend = 300
# ls_dtask = FindDTasks(ls_site,ls_spot,ls_shop,istart,iend,ls_dorder,CAPACITY)
# print 'total_worker is ',total_worker
# print 'total cost is ', cost
# print 'ls_dtask is ', ls_dtask
# ExportDTaskToCSV(ls_dtask,ls_site)
ls_dtask = []
ls_dtask = LoadDTaskFmCSV('../results/tempres/',ls_site,ls_spot,ls_shop)
ls_oSite = GroupTasks(ls_site,ls_spot,ls_shop,ls_dtask,ls_otoorder)

spotDic = {site.sid : (site.lat , site .lng) for site in ls_spot}
shopDic = {shop.sid : (shop.lat, shop.lng) for shop in ls_shop}





In [90]:
from xml.dom import minidom

def addService(services,newOrder):
    
    newService = doc.createElement('service')
    newService.setAttribute('id',newOrder['orderID'])
    newService.setAttribute('type','service')
    services.appendChild(newService)
    
    sLocationId = doc.createElement('locationId')
    newService.appendChild(sLocationId)
    sLocationId.appendChild(doc.createTextNode(newOrder['SId']))
    
    coord = doc.createElement('coord')
    newService.appendChild(coord)
    coord.setAttribute('x',newOrder['lat'])
    coord.setAttribute('y',newOrder['lng'])
    demand = doc.createElement('capacity-demand')
    newService.appendChild(demand)
    demand.appendChild(doc.createTextNode(newOrder['demand']))
    
    duration = doc.createElement('duration')
    newService.appendChild(duration)
    duration.appendChild(doc.createTextNode(newOrder['duration']))
    
    timeWs = doc.createElement('timeWindows')
    newService.appendChild(timeWs)
    timeWindow = doc.createElement('timeWindow')
    timeWs.appendChild(timeWindow)
    start = doc.createElement('start')
    timeWindow.appendChild(start)
    start.appendChild(doc.createTextNode(newOrder['start']))
    end = doc.createElement('end')
    timeWindow.appendChild(end)
    end.appendChild(doc.createTextNode(newOrder['end']))

def addO2OService(o2oOrders, newO2OService):
    newOrder = doc.createElement('shipment')
    o2oOrders.appendChild(newOrder)
    newOrder.setAttribute('id',newO2OService['orderID'])
    
    pickUp = doc.createElement('pickup')
    newOrder.appendChild(pickUp)
    pCoord =doc.createElement('coord')
    pickUp.appendChild(pCoord)
    pCoord.setAttribute('x',newO2OService['plat'])
    pCoord.setAttribute('y',newO2OService['plng'])
    
    pTimeWds = doc.createElement('timeWindows')
    pickUp.appendChild(pTimeWds)
    pTW = doc.createElement('timeWindow')
    pTimeWds.appendChild(pTW)
    
    pStart = doc.createElement('start')
    pStart.appendChild(doc.createTextNode(newO2OService['pStart']))
    pEnd = doc.createElement('end')
    pEnd.appendChild(doc.createTextNode(newO2OService['pEnd']))
    pTW.appendChild(pStart)
    pTW.appendChild(pEnd)
    
    delivery = doc.createElement('delivery')
    newOrder.appendChild(delivery)
    dCoord =doc.createElement('coord')
    delivery.appendChild(dCoord)
    dCoord.setAttribute('x',newO2OService['dlat'])
    dCoord.setAttribute('y',newO2OService['dlng'])
    
    dTimeWds = doc.createElement('timeWindows')
    delivery.appendChild(dTimeWds)
    dTW = doc.createElement('timeWindow')
    dTimeWds.appendChild(dTW)
    
    dStart = doc.createElement('start')
    dStart.appendChild(doc.createTextNode(newO2OService['dStart']))
    dEnd = doc.createElement('end')
    dEnd.appendChild(doc.createTextNode(newO2OService['dEnd']))
    dTW.appendChild(dStart)
    dTW.appendChild(dEnd)
    
    capacity_dimentions = doc.createElement('capacity-dimensions')
    newOrder.appendChild(capacity_dimentions)
    dimention = doc.createElement('dimension')
    capacity_dimentions.appendChild(dimention)
    dimention.setAttribute('index','0')
    dimention.appendChild(doc.createTextNode(newO2OService['capacity']))
       










def xmlGenerator (site, doc, ls_shipmentDic, ls_serviceDic):
    doc.toprettyxml(encoding='utf-8')
    problem = doc.createElement('problem')
    problem.setAttribute('xmlns','http://www.w3schools.com')
    problem.setAttribute('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
    problem.setAttribute('xsi:schemaLocation','http://www.w3schools.com vrp_xml_schema.xsd')
    doc.appendChild(problem)
    problemType = doc.createElement('problemType')
    problem.appendChild(problemType)
    fleetSize = doc.createElement('fleetSize')
    fleetSize.appendChild(doc.createTextNode("INFINITE"))
    problemType.appendChild(fleetSize)
    fleetComposition = doc.createElement('fleetComposition')
    fleetComposition.appendChild(doc.createTextNode('HOMOGENEOUS'))
    problemType.appendChild(fleetComposition)

    vehicles = doc.createElement('vehicles')
    problem.appendChild(vehicles)

    vehicle = doc.createElement('vehicle')
    vehicles.appendChild(vehicle)

    v_id = doc.createElement('id')
    vehicle.appendChild(v_id)
    v_id.appendChild(doc.createTextNode('Site'))

    v_typeId = doc.createElement('typeId')
    vehicle.appendChild(v_typeId)
    v_typeId.appendChild(doc.createTextNode("v8"))

    vLocation = doc.createElement('startLocation')
    vehicle.appendChild(vLocation)
    vlID = doc.createElement('id')
    vLocation.appendChild(vlID)
    vlID.appendChild(doc.createTextNode(site.sid))
    vCoord = doc.createElement('coord')
    vLocation.appendChild(vCoord)
    vCoord.setAttribute('x',str(site.lat))
    vCoord.setAttribute('y',str(site.lng))

    vtimeSchedule = doc.createElement('timeSchedule')
    vehicle.appendChild(vtimeSchedule)
    vStartT = doc.createElement('start')
    vStartT.appendChild(doc.createTextNode('0.0'))
    vEndT = doc.createElement('end')
    vEndT.appendChild(doc.createTextNode('999999.00'))
    vtimeSchedule.appendChild(vStartT)
    vtimeSchedule.appendChild(vEndT)

    vReturn2Site = doc.createElement('returnToDepot')
    vehicle.appendChild(vReturn2Site)
    vReturn2Site.appendChild(doc.createTextNode('true'))

    vehicleTypes = doc.createElement('vehicleTypes')
    problem.appendChild(vehicleTypes)
    vType = doc.createElement('type')
    vehicleTypes.appendChild(vType)
    vTypeId = doc.createElement('id')
    vType.appendChild(vTypeId)
    vTypeId.appendChild(doc.createTextNode('v8'))
    vCapacity = doc.createElement('capacity')
    vType.appendChild(vCapacity)
    vCapacity.appendChild(doc.createTextNode('140'))
    vCosts = doc.createElement('costs')
    vType.appendChild(vCosts)
    vCFixed = doc.createElement('fixed')
    vCosts.appendChild(vCFixed)
    vCFixed.appendChild(doc.createTextNode('0.0'))
    vCdistance = doc.createElement('distance')
    vCosts.appendChild(vCdistance)
    vCdistance.appendChild(doc.createTextNode('0.004'))
    vCTime = doc.createElement('time')
    vCosts.appendChild(vCTime)
    vCTime.appendChild(doc.createTextNode('0.0'))

    services = doc.createElement('services')
    problem.appendChild(services)
    
    o2oOrders =  doc.createElement('shipments')
    problem.appendChild(o2oOrders)
    
    for newService in ls_serviceDic:
        addService(services,newService)
    
    for newO2Oservice in ls_shipmentDic:
        addO2OService(o2oOrders,newO2Oservice)
    
    
    





for i  in range(0,1):
    [ls_spot_per_site,temp_order] = FindSpotPerSite(ls_site[i],ls_spot,ls_dorder)
    site  = ls_site[i]
    ls_o2o=ls_oSite[i].ls_oto
    #create develiries dictionary
    ls_servicesDic = []
    ls_o2oDic = []
    doc = minidom.Document()
    for j in range(0,1):
        spotId = ls_spot_per_site[j].sid
        position = spotDic.get(spotId)
        demand = temp_order.get(spotId)
        ls_servicesDic.append({"orderID":str(j), "SId":spotId, "lat":str(position[0]),"lng":str(position[1]),\
            "demand":str(demand),"duration":str(loadTime(demand)),"start":"0.0","end":"999999"})
    for k in range(0,1):
        duration = loadTime(ls_o2o[k][0].num)
        o2oOrder = ls_o2o[k][0]
        pPosition = spotDic.get(o2oOrder.spotid)
        dPostion = shopDic.get(o2oOrder.shopid)
        ls_o2oDic.append({"orderID":o2oOrder.oid,"plat":str(pPosition[0]),"plng":str(pPosition[1]),"pStart":str(o2oOrder.ptime),\
                         "pEnd":str(o2oOrder.ptime+duration),"dlat":str(dPostion[0]),"dlng":str(dPostion[1]),\
                         "dStart":str(o2oOrder.ptime+5),"dEnd":str(o2oOrder.dtime),"capacity":str(o2oOrder.num)})
    xmlGenerator (site, doc, ls_o2oDic, ls_servicesDic)

    fileName = site.sid + "_problem"
    fXML = file(fileName,"wb")
    doc.writexml(fXML,'\t','\t','\n',"utf-8")
    fXML.close()
    print site.sid + " done!\n"

A001 done!



In [83]:
{"orderID":str(j), "SId":spotId, "lat":str(position[0]),"lng":str(position[1]),\
            "demand":str(demand),"duration":str(loadTime(demand)),"start":"0.0","end":"999999"}


{'SId': 'B0214',
 'demand': '21',
 'duration': '18.7477270849',
 'end': '999999',
 'lat': '31.268236',
 'lng': '121.481796',
 'orderID': '0',
 'start': '0.0'}